<a href="https://colab.research.google.com/github/MohamedKhalidmk/Supervised_Learning/blob/main/Logistic_Regression_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

In [113]:
rice_dataset_raw = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/Rice_Cammeo_Osmancik.csv")

print("Dataset Summary Statistics:\n" + "-" * 40)
print(rice_dataset_raw.describe())

# Split into features and target
X = rice_dataset_raw.drop(columns=['Class'])
y = rice_dataset_raw['Class']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=2
)

print("\nData Shapes:\n" + "-" * 40)
print(f"{'X_train shape':<15}: {X_train.shape}")
print(f"{'X_test shape' :<15}: {X_test.shape}")
print(f"{'y_train shape':<15}: {y_train.shape}")
print(f"{'y_test shape' :<15}: {y_test.shape}")

print("\nData Types:\n" + "-" * 40)
print("Features:")
print(X.dtypes)
print("\nTarget:")
print(y.dtypes)

# Encode labels (Cammeo/Osmancik → 0/1)
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

print("\nLabel Encoding Map:\n" + "-" * 40)
for class_name, class_id in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
    print(f"{class_name:<10} → {class_id}")

Dataset Summary Statistics:
----------------------------------------
               Area    Perimeter  Major_Axis_Length  Minor_Axis_Length  \
count   3810.000000  3810.000000        3810.000000        3810.000000   
mean   12667.727559   454.239180         188.776222          86.313750   
std     1732.367706    35.597081          17.448679           5.729817   
min     7551.000000   359.100006         145.264465          59.532406   
25%    11370.500000   426.144753         174.353855          82.731695   
50%    12421.500000   448.852493         185.810059          86.434647   
75%    13950.000000   483.683746         203.550438          90.143677   
max    18913.000000   548.445984         239.010498         107.542450   

       Eccentricity   Convex_Area       Extent  
count   3810.000000   3810.000000  3810.000000  
mean       0.886871  12952.496850     0.661934  
std        0.020818   1776.972042     0.077239  
min        0.777233   7723.000000     0.497413  
25%        0.872402

In [114]:
results = {}
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [115]:
class Batch_Gradient_Descent:
  def __init__(self, learning_rate=0.001, n_iters=1000):
    self.lr = learning_rate
    self.n_iters = n_iters
  def sigmoid(self, x):
    return 1/(1+np.exp(-x))
  def loss(self, y, y_pred):
    return (-y*np.log(y_pred) - (1-y)*np.log(1-y_pred)).mean()
  def fit(self, X, y):
    self.weights =np.zeros(X.shape[1])
    self.bias = 0
    self.losses = []
    for i in range(self.n_iters):
      z_pred = np.dot(X, self.weights) + self.bias
      y_pred = self.sigmoid(z_pred)
      dw = (1/X.shape[0])*np.dot(X.T, (y_pred-y))
      db = (1/X.shape[0])*np.sum(y_pred-y)
      self.weights -= self.lr*dw
      self.bias -= self.lr*db
      self.losses.append(self.loss(y, y_pred))
      print(f"epoch:{i+1}/{self.n_iters}, loss:{self.losses[-1]}")
  def predict(self, X):
    z_pred = np.dot(X, self.weights) + self.bias
    y_pred = self.sigmoid(z_pred)
    y_pred = np.where(y_pred>0.5, 1, 0)
    return y_pred

In [116]:
class Mini_batch_gradient_descent:
  def __init__(self, learning_rate=0.001, n_iters=1000, batch_size=32):
    self.lr = learning_rate
    self.n_iters = n_iters
    self.batch_size = batch_size
  def sigmoid(self, x):
    return 1/(1+np.exp(-x))
  def loss(self, y, y_pred):
    return (-y*np.log(y_pred) - (1-y)*np.log(1-y_pred)).mean()
  def fit(self, X, y):
    self.weights =np.zeros(X.shape[1])
    self.bias = 0
    self.losses = []
    n_samples=X.shape[0]
    for i in range(self.n_iters):
     indices = np.arange(n_samples)
     np.random.shuffle(indices)
     X = X[indices]
     y = y[indices]
     for j in range(0, n_samples, self.batch_size):
       X_batch = X[j:j+self.batch_size]
       y_batch = y[j:j+self.batch_size]
       z_pred = np.dot(X_batch, self.weights) + self.bias
       y_pred = self.sigmoid(z_pred)
       dw = (1/self.batch_size)*np.dot(X_batch.T, (y_pred-y_batch))
       db = (1/self.batch_size)*np.sum(y_pred-y_batch)
       self.weights -= self.lr*dw
       self.bias -= self.lr*db
       self.losses.append(self.loss(y_batch, y_pred))
     print(f"epoch:{i+1}/{self.n_iters}, loss:{self.losses[-1]}")
  def predict(self, X):
    y_pred = np.dot(X, self.weights) + self.bias
    y_pred = self.sigmoid(y_pred)
    y_pred = np.where(y_pred>0.5, 1, 0)
    return y_pred

In [117]:
class Stochastic_gradient_descent:
  def __init__(self, learning_rate=0.001, n_iters=1000):
    self.lr = learning_rate
    self.n_iters = n_iters
  def sigmoid(self, x):
    return 1/(1+np.exp(-x))
  def loss(self, y, y_pred):
    return (-y*np.log(y_pred) - (1-y)*np.log(1-y_pred)).mean()
  def fit(self, X, y):
    self.weights =np.zeros(X.shape[1])
    self.bias = 0
    self.losses = []
    n_samples=X.shape[0]
    for i in range(self.n_iters):
     indices = np.arange(n_samples)
     np.random.shuffle(indices)
     X_shuffled = X[indices]
     y_shuffled= y[indices]
     for j in range(n_samples):
       X_j = X_shuffled[j].reshape(1, -1)
       y_j = y_shuffled[j]
       z_pred = np.dot(X_j, self.weights) + self.bias
       y_pred = self.sigmoid(z_pred)
       dw = np.dot(X_j.T, (y_pred-y_j))
       db = np.sum(y_pred-y_j)
       self.weights -= self.lr*dw
       self.bias -= self.lr*db
       self.losses.append(self.loss(y_j, y_pred))
     print(f"epoch:{i+1}/{self.n_iters}, loss:{self.losses[-1]}")
  def predict(self, X):
    z_pred = np.dot(X, self.weights) + self.bias
    y_pred = self.sigmoid(z_pred)
    y_pred = np.where(y_pred>0.5, 1, 0)
    return y_pred

In [118]:
model1 = SGDClassifier(loss='log_loss', learning_rate='constant', eta0=0.01, max_iter=1, random_state=42)

print("Training Progress:\n" + "-" * 30)

# Epoch-wise training
for i in range(10):
    model1.partial_fit(X_train, y_train, classes=np.unique(y_train))
    y_pred = model1.predict(X_test)
    loss = log_loss(y_test, y_pred)
    print(f"Epoch {i+1:>2}: loss = {loss:.4f}")

# Final predictions and metrics
y_pred = model1.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\nFinal Confusion Matrix:\n" + "-" * 30)
print(cm)

print("\nFinal Evaluation Metrics:\n" + "-" * 30)
print(f"{'Accuracy' : <10}: {acc:.4f}")
print(f"{'Precision': <10}: {prec:.4f}")
print(f"{'Recall'   : <10}: {rec:.4f}")
print(f"{'F1 Score' : <10}: {f1:.4f}")

Training Progress:
------------------------------
Epoch  1: loss = 2.8381
Epoch  2: loss = 2.5070
Epoch  3: loss = 2.5070
Epoch  4: loss = 2.5070
Epoch  5: loss = 2.5070
Epoch  6: loss = 2.5070
Epoch  7: loss = 2.5070
Epoch  8: loss = 2.5070
Epoch  9: loss = 2.5070
Epoch 10: loss = 2.5070

Final Confusion Matrix:
------------------------------
[[296  25]
 [ 28 413]]

Final Evaluation Metrics:
------------------------------
Accuracy  : 0.9304
Precision : 0.9429
Recall    : 0.9365
F1 Score  : 0.9397


In [119]:
model2 = Batch_Gradient_Descent(learning_rate=0.01, n_iters=10)
model2.fit(X_train, y_train)

# Predict
y_pred = model2.predict(X_test)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n" + "-" * 30)
print(cm)

# Evaluation Metrics
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\nEvaluation Metrics:\n" + "-" * 30)
print(f"{'Accuracy' :<10}: {acc:.4f}")
print(f"{'Precision':<10}: {prec:.4f}")
print(f"{'Recall'   :<10}: {rec:.4f}")
print(f"{'F1 Score' :<10}: {f1:.4f}")
results['BGD'] = acc

epoch:1/10, loss:0.6931471805599454
epoch:2/10, loss:0.6858825173690675
epoch:3/10, loss:0.6787790919882983
epoch:4/10, loss:0.6718331982504676
epoch:5/10, loss:0.6650411664117513
epoch:6/10, loss:0.6583993676143554
epoch:7/10, loss:0.651904217933616
epoch:8/10, loss:0.6455521820202604
epoch:9/10, loss:0.6393397763506867
epoch:10/10, loss:0.6332635720998794

Confusion Matrix:
------------------------------
[[292  29]
 [ 42 399]]

Evaluation Metrics:
------------------------------
Accuracy  : 0.9068
Precision : 0.9322
Recall    : 0.9048
F1 Score  : 0.9183


In [120]:
# Fit the model
model3 = Mini_batch_gradient_descent(learning_rate=0.01, n_iters=10, batch_size=2)
model3.fit(X_train, y_train)

# Predict
y_pred = model3.predict(X_test)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n" + "-" * 30)
print(cm)

# Evaluation Metrics
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\nEvaluation Metrics:\n" + "-" * 30)
print(f"{'Accuracy' :<10}: {acc:.4f}")
print(f"{'Precision':<10}: {prec:.4f}")
print(f"{'Recall'   :<10}: {rec:.4f}")
print(f"{'F1 Score' :<10}: {f1:.4f}")
results['MBGD'] = acc

epoch:1/10, loss:0.21557233375210538
epoch:2/10, loss:0.04940319479721092
epoch:3/10, loss:0.001240656657218024
epoch:4/10, loss:1.2735230613861122
epoch:5/10, loss:0.03894144054308221
epoch:6/10, loss:0.0014969339504781719
epoch:7/10, loss:0.026516330723311096
epoch:8/10, loss:0.028852280197796336
epoch:9/10, loss:0.20777594299076096
epoch:10/10, loss:0.002554120481593923

Confusion Matrix:
------------------------------
[[296  25]
 [ 28 413]]

Evaluation Metrics:
------------------------------
Accuracy  : 0.9304
Precision : 0.9429
Recall    : 0.9365
F1 Score  : 0.9397


In [121]:
# Fit the model
model4 = Stochastic_gradient_descent(learning_rate=0.01, n_iters=10)
model4.fit(X_train, y_train)

# Predict
y_pred = model4.predict(X_test)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n" + "-" * 30)
print(cm)

# Evaluation Metrics
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\nEvaluation Metrics:\n" + "-" * 30)
print(f"{'Accuracy' :<10}: {acc:.4f}")
print(f"{'Precision':<10}: {prec:.4f}")
print(f"{'Recall'   :<10}: {rec:.4f}")
print(f"{'F1 Score' :<10}: {f1:.4f}")
results['SGD'] = acc

epoch:1/10, loss:0.5236523275490248
epoch:2/10, loss:0.0010527369674178427
epoch:3/10, loss:0.08749333241809706
epoch:4/10, loss:0.3324791757638775
epoch:5/10, loss:0.003908255259712406
epoch:6/10, loss:0.00161815174552399
epoch:7/10, loss:0.08409233048960528
epoch:8/10, loss:0.19169220907806364
epoch:9/10, loss:0.000299622170702654
epoch:10/10, loss:0.2350090399321735

Confusion Matrix:
------------------------------
[[296  25]
 [ 28 413]]

Evaluation Metrics:
------------------------------
Accuracy  : 0.9304
Precision : 0.9429
Recall    : 0.9365
F1 Score  : 0.9397


In [122]:
print("\nModel Comparison (Accuracy):")
print("-" * 40)
for model_name, accuracy in results.items():
    print(f"{model_name:<10}: {accuracy:.4f}")


Model Comparison (Accuracy):
----------------------------------------
BGD       : 0.9068
MBGD      : 0.9304
SGD       : 0.9304
